In [1]:
!pip install datasets transformers tensorflow numpy matplotlib opencv-python scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.3 MB/s eta 0:00:00


In [2]:
import numpy as np
from PIL import Image
from datasets import load_dataset

# تحميل مجموعة البيانات
dataset = load_dataset("yasimed/lung_cancer")

# دالة استخراج التصنيفات من النصوص
def extract_label(text):
    return 1 if "cancer" in text.lower() else 0  # تحويل النص إلى تصنيف رقمي

# استخراج الصور والتصنيفات مع التأكد من الاتساق
images = []
labels = []

for item in dataset['train']:
    # تحويل الصورة إلى كائن PIL
    image = item['image'].convert("RGB")  # تحويل الصورة إلى 3 قنوات (RGB)

    # تغيير الحجم إلى 224x224
    image = image.resize((224, 224))

    # تحويل الصورة إلى مصفوفة NumPy وتطبيع القيم بين 0 و 1
    image = np.array(image, dtype=np.float32) / 255.0

    images.append(image)
    labels.append(extract_label(item['text']))  # استخراج التصنيف من النص

# تحويل القوائم إلى مصفوفات NumPy بعد التأكد من التجانس
images = np.array(images, dtype=np.float32)
labels = np.array(labels, dtype=np.int32)

# التحقق من الأشكال
print("✅ شكل الصور:", images.shape)  # يجب أن يكون (613, 224, 224, 3)
print("✅ شكل التصنيفات:", labels.shape)  # يجب أن يكون (613,)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/313 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/59.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/613 [00:00<?, ? examples/s]

✅ شكل الصور: (613, 224, 224, 3)
✅ شكل التصنيفات: (613,)


In [5]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


In [3]:
from sklearn.model_selection import train_test_split

# تقسيم البيانات إلى تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# طباعة حجم البيانات بعد التقسيم
print("✅ عدد صور التدريب:", X_train.shape[0])
print("✅ عدد صور الاختبار:", X_test.shape[0])


✅ عدد صور التدريب: 490
✅ عدد صور الاختبار: 123


In [6]:
# إنشاء محول بيانات التدريب
train_datagen = ImageDataGenerator(
    rotation_range=20,  # تدوير عشوائي حتى 20 درجة
    width_shift_range=0.1,  # إزاحة أفقية بنسبة 10%
    height_shift_range=0.1,  # إزاحة رأسية بنسبة 10%
    horizontal_flip=True,  # قلب الصورة أفقيًا
)

# تجهيز البيانات للتدريب والاختبار
train_generator = train_datagen.flow(X_train, y_train, batch_size=32, shuffle=True)
test_generator = ImageDataGenerator().flow(X_test, y_test, batch_size=32, shuffle=False)


In [7]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

def residual_block(x, filters, expansion=4):
    """
    كتلة متبقية موسعة مستوحاة من ResNet مع تعديلات
    """
    shortcut = x
    x = Conv2D(filters, (1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # محاكاة نمط VGG مع 3x3 convolutions متتالية
    for _ in range(2):
        x = Conv2D(filters, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

    # توسيع القنوات لمطابقة الأبعاد
    x = Conv2D(filters*expansion, (1, 1), padding='same')(x)
    x = BatchNormalization()(x)

    # مطابقة أبعاد الوصلة التخطية
    if shortcut.shape[-1] != filters*expansion:
        shortcut = Conv2D(filters*expansion, (1, 1), padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

def hybrid_model(input_shape=(224, 224, 3), num_classes=1):
    inputs = Input(shape=input_shape)

    # المرحلة الأولى: معالجة أولية
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # مجموعة الكتل الهجينة
    filters = [64, 128, 256, 512]
    repetitions = [3, 4, 6, 3]  # إجمالي 16 كتلة

    for i, (f, r) in enumerate(zip(filters, repetitions)):
        for j in range(r):
            strides = (2, 2) if (j == 0 and i > 0) else (1, 1)
            if strides != (1, 1):
                x = Conv2D(f, (1, 1), strides=strides)(x)
                x = BatchNormalization()(x)
            x = residual_block(x, f)

    # الطبقات النهائية
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    return model

# إنشاء النموذج
model = hybrid_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# عرض ملخص النموذج
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 112, 112, 64)   │          9,472 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 112, 112, 64)   │            256 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu (ReLU)              │ (None, 112, 112, 64)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 56, 56, 64)     │              0 │ re_lu[0][0]            │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 56, 56, 64)     │          4,160 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 56, 56, 64)     │            256 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_1 (ReLU)            │ (None, 56, 56, 64)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 56, 56, 64)     │         36,928 │ re_lu_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 56, 56, 64)     │            256 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_2 (ReLU)            │ (None, 56, 56, 64)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 56, 56, 64)     │         36,928 │ re_lu_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 56, 56, 64)     │            256 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_3 (ReLU)            │ (None, 56, 56, 64)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 56, 56, 256)    │         16,640 │ re_lu_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 56, 56, 256)    │         16,640 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 56, 56, 256)    │          1,024 │ conv2d_4[0][0]         │
│ (BatchNormalization) 

 Total params: 36,519,617 (139.31 MB)

 Trainable params: 36,457,153 (139.07 MB)

 Non-trainable params: 62,464 (244.00 KB)

In [8]:
# تدريب النموذج
history = model.fit(train_generator, validation_data=test_generator, epochs=40)

# حفظ النموذج بعد التدريب
model.save("CUSTOM.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 121s 3s/step - accuracy: 0.7854 - loss: 1.5408 - val_accuracy: 0.2195 - val_loss: 99.1674
Epoch 2/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 475ms/step - accuracy: 0.8934 - loss: 0.6822 - val_accuracy: 0.2195 - val_loss: 6.6790
Epoch 3/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 443ms/step - accuracy: 0.8934 - loss: 0.4730 - val_accuracy: 0.2195 - val_loss: 78.2321
Epoch 4/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 505ms/step - accuracy: 0.8976 - loss: 0.6522 - val_accuracy: 0.2195 - val_loss: 2.4883
Epoch 5/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 469ms/step - accuracy: 0.9333 - loss: 0.3456 - val_accuracy: 0.1870 - val_loss: 0.7097
Epoch 6/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 459ms/step - accuracy: 0.8581 - loss: 0.3586 - val_accuracy: 0.2195 - val_loss: 0.7684
Epoch 7/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 488ms/step - accuracy: 0.9104 - loss: 0.4241 - val_accuracy: 0.7805 - val_loss: 1.2663
Epoch 8/40
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 452ms/step - accuracy: 0.9164 - loss: 0.4092 - val_accuracy: 0

In [12]:
# تقييم النموذج
loss, accuracy = model.evaluate(test_generator)
print(f"✅ دقة النموذج على بيانات الاختبار: {accuracy * 100:.2f}%")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.9294 - loss: 0.1443
✅ دقة النموذج على بيانات الاختبار: 95.12%


In [18]:
import numpy as np
from PIL import Image

def preprocess_image(image_path):
    """ تحميل الصورة وتحويلها إلى الشكل المناسب للنموذج """
    image = Image.open(image_path).convert("RGB")  # تحويل إلى 3 قنوات (RGB)
    image = image.resize((224, 224))  # تغيير الحجم إلى 224x224
    image = np.array(image, dtype=np.float32) / 255.0  # تطبيع القيم بين 0 و 1
    image = np.expand_dims(image, axis=0)  # إضافة بعد جديد ليتناسب مع الإدخال المتوقع (batch_size, 224, 224, 3)
    return image

# تجربة الصورة
image_path = "/content/image.png"  # ضع مسار الصورة هنا
processed_image = preprocess_image(image_path)

# التنبؤ باستخدام النموذج
prediction = model.predict(processed_image)

# تفسير النتيجة بناءً على النص الموجود في البيانات
def interpret_prediction(pred):
    """ تحويل التنبؤ الرقمي إلى نص بشري """
    if pred >= 0.5:
        return "مصاب بسرطان الرئة 🩸"
    else:
        return "غير مصاب بالسرطان ✅"

# عرض النتيجة النصية مع احتمالية الإصابة
result = interpret_prediction(prediction[0][0])
print(f"🔍 النتيجة: {result} (احتمال الإصابة: {prediction[0][0] * 100:.2f}%)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
🔍 النتيجة: غير مصاب بالسرطان ✅ (احتمال الإصابة: 34.12%)
